<a href="https://colab.research.google.com/github/ChristianConchari/Strathospheric-ballon-earth-observation-gathered-imagery-classification-through-TinyML/blob/master/classification_model_ballon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow import keras
from tensorflow.keras import regularizers
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
!unzip /content/gdrive/MyDrive/data_training_balloon.zip
!mkdir data_training_balloon 
!mv 0 data_training_balloon
!mv 1 data_training_balloon

In [ ]:
datagenerator = ImageDataGenerator(
        rescale = 1/255.0,
        rotation_range=30, # rotation
        width_shift_range=0.2, # horizontal shift
        height_shift_range=0.2, # vertical shift
        zoom_range=0.3, # zoom
        horizontal_flip=True, # horizontal flip
        vertical_flip=True, # horizontal flip
        validation_split = 0.2) 

In [ ]:
training_data = '/content/data_training_balloon'

img_height = 100
img_width = 100
batch_size = 16

train_ds = datagenerator.flow_from_directory(
    training_data,
    color_mode='rgb',
    class_mode = "binary",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle = True,
    subset = "training"
)

val_ds = datagenerator.flow_from_directory(
    training_data,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle = True,
    color_mode='rgb',
    class_mode = "binary",
    subset = "validation"
)

In [ ]:
def plotImages(images_arr):
  fig, axes = plt.subplots(1, 4, figsize = (10, 10))
  axes = axes.flatten()
  for img, ax in zip(images_arr, axes):
    ax.imshow(img)
    ax.axis("off")
  plt.tight_layout()
  plt.show()

In [ ]:
sample_training_images, _= next(iter(train_ds))
print(sample_training_images.shape)
plotImages(sample_training_images[:4])

In [ ]:
model_cnn = keras.models.Sequential()

model_cnn.add(keras.layers.Conv2D(16, (3,3), padding='same'))
model_cnn.add(keras.layers.Activation(keras.activations.relu))
model_cnn.add(keras.layers.BatchNormalization())
model_cnn.add(keras.layers.MaxPooling2D((2,2)))

model_cnn.add(keras.layers.Conv2D(32, (3,3), padding='same'))
model_cnn.add(keras.layers.Activation(keras.activations.relu))
model_cnn.add(keras.layers.BatchNormalization())
model_cnn.add(keras.layers.MaxPooling2D((2,2)))

model_cnn.add(keras.layers.Conv2D(64, (3,3), padding='same', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model_cnn.add(keras.layers.Activation(keras.activations.relu))
model_cnn.add(keras.layers.BatchNormalization())
model_cnn.add(keras.layers.MaxPooling2D((2,2)))

model_cnn.add(keras.layers.Conv2D(128, (3,3), padding='same', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model_cnn.add(keras.layers.Activation(keras.activations.relu))
model_cnn.add(keras.layers.BatchNormalization())
model_cnn.add(keras.layers.MaxPooling2D((2,2)))

model_cnn.add(keras.layers.Flatten())
model_cnn.add(keras.layers.Dense(128,  kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model_cnn.add(keras.layers.Dropout(0.5))
model_cnn.add(keras.layers.Activation(keras.activations.relu))
model_cnn.add(keras.layers.Dense(1, activation = "sigmoid"))

model_cnn.build((None, 100, 100, 3))
model_cnn.summary()

In [ ]:
optimizer = keras.optimizers.SGD(
    learning_rate=0.0001, momentum=0.9, nesterov=True, name="SGD"
)

model_cnn.compile(optimizer=optimizer, loss = "binary_crossentropy", metrics = ["accuracy"])
training_samples = train_ds.n
val_samples = val_ds.n

epochs = 30

history_cnn = model_cnn.fit(
    train_ds,
    steps_per_epoch=training_samples // batch_size,
    epochs = epochs,
    validation_data=val_ds,
    validation_steps=val_samples // batch_size,
)

In [ ]:
def visualizacion_resultados(history):
  nepochs = [i for i in range(epochs)]
  fig, ax = plt.subplots(1,2)
  train_acc = history.history["accuracy"]
  train_loss = history.history["loss"]
  val_acc = history.history["val_accuracy"]
  val_loss = history.history["val_loss"]
  fig.set_size_inches(16, 9)

  ax[0].plot(nepochs, train_acc, "-", label =" Training accuracy")
  ax[0].plot(nepochs, val_acc, "-", label= "Validation accuracy")
  ax[0].set_title("Training & validation accuracy")
  ax[0].legend()
  ax[0].set_xlabel("nEpochs")
  ax[0].set_ylabel("Accuracy")

  ax[1].plot(nepochs, train_loss, "go-", label =" Training Loss")
  ax[1].plot(nepochs, val_loss, "ro-", label= "Validation Loss")
  ax[1].set_title("Training & validation Loss")
  ax[1].legend()
  ax[1].set_xlabel("Epochs")
  ax[1].set_ylabel("Loss")

  plt.show()


In [ ]:
visualizacion_resultados(history_cnn)

In [ ]:
loss, accuracy = model_cnn.evaluate(val_ds)

In [ ]:
sample_training_images, _= next(iter(train_ds))
print(sample_training_images.shape)
plotImages(sample_training_images[:4])
model_cnn(sample_training_images[:4])